In [53]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.lines as mlines
from IPython.display import display, Latex

In [60]:
A = np.array([[-1,0],[0,-1],[-1,1]]) # coefficient matrix for linear constraints
b = np.array([0,0,0]) # solution vector for linear constraints
# x = [r0,ri]
def obj(x):
    """
    function to evaluate objective function
    """
    c = (1150*2*50*50/(0.4*1194000*3.14))
    rc = 0.04
    omega = (50/0.381)
    d = 3.3*(7250*(omega**2)/8)
    r = 0.0076
    sigma_y = 200000000
    s = 0.005

    f = c/((x[0]**2)-(x[1]**2))
    dfdx = np.array([-c*2*x[0]/((x[0]**2) -(x[1]**2))**2
            ,2*c*x[1]/((x[0]**2)-(x[1]**2))**2]) # it should an n-element vector

    """
    in case of simulation:
    write x to a text file or as command line arguments
    call/run simulation (either reads the text file or accepts arguments)
    read results from a text file (output by the simulation)
    f = ...
    see examples in 9_dfo
    """

    return f, dfdx

def nonlincon(x):
    """
    function that evaluates general nonlinear constraints
    """
    c = (1150*2*50*50/(0.4*1194000*3.14))
    rc = 0.04
    omega = (50/(0.3810*0.5))
    d = 3.3*(7250*(omega**2)/8)
    r = 0.0076
    sigma_y = 200000000
    s = 0.005
    # evaluate constraints
    g = np.array([d*((x[1]+s-rc)**2)-sigma_y, ]) # it should an m-element vector

    return g

def nonlinconJac(x):
    """
    function that evaluates gradients of general nonlinear constraints
    """
    c = (1150*2*50*50/(0.4*1194000*3.14))
    rc = 0.04
    omega = (50/(0.3810*0.5))
    d = 3.3*(7250*(omega**2)/8)
    r = 0.0076
    sigma_y = 200000000
    s = 0.005
    # evaluate constraint derivatives
    dgdx = np.array([[0, 2*d*(x[1]+s-rc)]]) # it should be a mxn array

    return dgdx

# for SLSQP constraints need to be passed to a list of dictionaries (see below)
# def lincon(x):
#     return (A @ x.reshape((-1,1)) - b.reshape((-1,1))).squeeze()

# def linconJac(x):
#     return A

In [61]:
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

x0 = np.array([0.11,0.07])
n = 2 # variables
m = 1 # nonlinear inequality constraint
bounds = [(0.099,0.12),(0.07,0.099)]

# choose an algorithm
method = "SLSQP"
# method = "trust-constr"

if method == "SLSQP":
    # for SLSQP constraints need to be passed as a list of dictionaries (read the documentation!!)

    constraints = []
    for A_i,b_i in zip(A,b):
        lin_cstr = {
            "type": "ineq", # ‘eq’ for equality, ‘ineq’ for inequality.
            "fun": lambda x : A_i @ x - b_i, # The function defining the constraint.
            "jac": lambda x : A_i, # The Jacobian of fun (only for SLSQP).
        }
        constraints += [lin_cstr]

    # lin_cstr = {
    #     "type": "ineq", # ‘eq’ for equality, ‘ineq’ for inequality.
    #     "fun": lincon, # The function defining the constraint.
    #     "jac": linconJac, # The Jacobian of fun (only for SLSQP).
    # }
    # constraints += [lin_cstr]


    nonlin_cstr = {
        "type": "ineq", # ‘eq’ for equality, ‘ineq’ for inequality.
        "fun": nonlincon, # The function defining the constraint.
        "jac": nonlinconJac, # The Jacobian of fun (only for SLSQP).
    }
    constraints += [nonlin_cstr]

elif method == "trust-constr":
    # for trust-constr constraints is a list of constraint objects (linear or nonlinear)
    lin_cstr = LinearConstraint(A,lb=-np.inf,ub=b)
    nonlin_cstr = NonlinearConstraint(nonlincon,lb=-np.inf,ub=np.zeros(m),jac=nonlinconJac)
    constraints = [lin_cstr,nonlin_cstr]

opt = minimize(obj, x0, args=(), method=method, jac=True, hess=None, hessp=None, 
    bounds=bounds, constraints=constraints, tol=1e-10, callback=None, options=None)

print("===========================")
display(Latex(r"$f(\mathbf{x}^*) = %.4f$" %(opt.fun)))
display(Latex(r"$\mathbf{x}^*$ = $[%.4f~~%.4f]^\mathrm{T}$" %(opt.x[0],opt.x[1])))

if method == "trust-constr":
    # evaluate constraints
    g1_opt = opt.constr[1][0]
    g2_opt = opt.constr[0][0] - b[0]
    g3_opt = opt.constr[0][1] - b[1]
    g4_opt = opt.constr[0][2] - b[2]
    # display(Latex(r"$\mathbf{g}(\mathbf{x}^*)$ = $[%.4f~~%.4f~~%.4f~~%.4f]^\mathrm{T}$" %(g1_opt,g2_opt,g3_opt,g4_opt)))
    print(g1_opt)
    print(g2_opt)
    print(g3_opt)
    print(g4_opt)
    print(opt.x[0])
    print(opt.x[1])
    print(opt.nit)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [4]:
from IPython.display import display, Latex
from scipy.optimize import minimize

In [10]:
#x = [R,r0,ri]
def obj(x):
    """
    The objective
    """
    c = (1150*2*50*50/(0.4*1194000*3.14))
    rc = 0.04
    omega = (50/0.381)
    d = 3.3*(7250*(omega**2)/8)
    r = 0.0076
    sigma_y = 200000000
    s = 0.005
    return ((c/((x[1]**2)-(x[2]**2))) - rk*( (1/(d*((x[1]+s-rc)**2)-sigma_y)) + (1/(x[2]-x[0])) + (1/(x[1]-x[0])) + (1/(x[2]-x[1])) + (1/x[2]) + (1/x[1]) + (1/x[0]))) 

In [13]:
def grad(x):
    """
    The gradient
    """
    c = (1150*2*50*50/(0.4*1194000*3.14))
    rc = 0.04
    omega = (50/0.381)
    d = 3.3*(7250*(omega**2)/8)
    r = 0.0076
    sigma_y = 200000000
    s = 0.005
    return [-rk*((1/(x[2]-x[0])**2) + (1/(x[1]-x[0])**2) - (1/(x[0]**2))),
            (-2*c*x[1]/((x[1]**2)-(x[2]**2))**2) + rk*(((2*d*(x[1]+s-rc))/((d*((x[1]+s-rc)**2)-sigma_y)**2))- (1/((x[2]-x[1])**2)) +(1/(x[1]-x[0])**2) +(1/x[1]**2)),
            (2*c*x[2]/((x[1]**2)-[x[2]**2])**2) + rk*((1/(x[2]-x[0])**2) + (1/(x[2]-x[1])**2) + (1/(x[2]**2)))]

In [14]:
r = [10.0, 1.0, 0.1, 0.01, 0.001] # sequence of decreasing r"s
r = r[0:5]
x0 = [0.10, 0.099, 0.07]
for k,rk in enumerate(r):
    x_opt = minimize(obj,x0,jac=grad,method="BFGS",options={"gtol": 1e-8, "maxiter": 1000, "disp": False})
    f_opt = obj(x_opt.x)

    # optimizer
    x0 = x_opt.x

    print("===========================")
    display(Latex(r"$k = %i$" %(k+1)))
    display(Latex(r"$\mathbf{x}_k^*$ = $[%.4f~,%.4f~,%.4f~]^\mathrm{T}$" %(x_opt.x[0],x_opt.x[1],x_opt.x[2])))


c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
c:\Users\vrajp\python_stuff\venvs\vraj_env\lib\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [6]:
print(f_opt)

[-3.1718717e+08]
